## Reference
- https://github.com/IBM/rag-chunking-techniques/blob/main/simple-rag.ipynb

In [1]:
!pip install "ibm-watson-machine-learning>=1.0.320" 
!pip install "pydantic>=1.10.0" 
!pip install langchain 
!pip install huggingface
!pip install huggingface-hub
!pip install sentence-transformers
!pip install chromadb


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
  Using cached sentence_transformers-2.2.2-py3-none-any.whl

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
import os
load_dotenv()


True

In [3]:
project_id = os.environ["PROJECT_ID"]

In [4]:
!pip install wget
import wget

filename = 'companyPolicies.txt'
url = 'https://raw.github.com/ravisrirangam/chunking_techniques/main/data/companypolicies.txt'

wget.download(url, out=filename)
print('file downloaded')

  Using cached wget-3.2-py3-none-any.whl

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
file downloaded


In [5]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

loader = TextLoader(filename)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
print(len(texts))

Created a chunk of size 1624, which is longer than the specified 1000
Created a chunk of size 1885, which is longer than the specified 1000
Created a chunk of size 1903, which is longer than the specified 1000
Created a chunk of size 1729, which is longer than the specified 1000
Created a chunk of size 1678, which is longer than the specified 1000
Created a chunk of size 2032, which is longer than the specified 1000
Created a chunk of size 1894, which is longer than the specified 1000


16


In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
import tqdm as notebook_tqdm

embeddings = HuggingFaceEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)
print('documents ingested')

documents ingested


In [8]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

model_id = ModelTypes.FLAN_UL2

In [9]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 130,
    GenParams.MAX_NEW_TOKENS: 200
}

In [12]:
from ibm_watson_machine_learning.foundation_models import Model

def get_credentials(api_key):
    return {
        "url" : "https://us-south.ml.cloud.ibm.com",
        "apikey" : api_key,
    }

model = Model(
    model_id=model_id,
    params=parameters,
    credentials=get_credentials(os.environ["IAM_API_KEY"]),
    project_id=project_id
)

In [13]:
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

flan_ul2_llm = WatsonxLLM(model=model)

In [14]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=flan_ul2_llm, chain_type="stuff", retriever=docsearch.as_retriever())
query = "mobile policy"
qa.run(query)

'Yes, it helps employees use mobile devices in a responsible way. It also helps the company by ensuring legal and ethical standards are met. It also helps the company by promoting the responsible and secure use of mobile devices. It also helps the company by ensuring ongoing alignment with evolving technology and security best practices. It also helps the company by promoting the responsible and secure use of mobile devices in line with legal and ethical standards. Every employee is expected to comprehend and abide by these guidelines. Regular reviews of the policy ensure its ongoing alignment with evolving technology and security best practices. It also helps the company by promoting the responsible and secure use of mobile devices in line with legal and ethical standards. Every employee is expected to comprehend and abide by these guidelines. Regular reviews of the policy ensure its ongoing alignment with evolving technology and security best practices.'

In [15]:
query = "mobile policy"
docs = docsearch.similarity_search(query)
print(len(docs))
for i in range(len(docs)):
    print(docs[i].page_content)
    print('\n\n')

4
4.	Mobile Phone Policy



4.	Mobile Phone Policy



The Mobile Phone Policy sets forth the standards and expectations governing the appropriate and responsible usage of mobile devices in the organization. The purpose of this policy is to ensure that employees utilize mobile phones in a manner consistent with company values and legal compliance.
Acceptable Use: Mobile devices are primarily intended for work-related tasks. Limited personal usage is allowed, provided it does not disrupt work obligations.
Security: Safeguard your mobile device and access credentials. Exercise caution when downloading apps or clicking links from unfamiliar sources. Promptly report security concerns or suspicious activities related to your mobile device.
Confidentiality: Avoid transmitting sensitive company information via unsecured messaging apps or emails. Be discreet when discussing company matters in public spaces.
Cost Management: Keep personal phone usage separate from company accounts and reimburse 